# Import Dataset

In [5]:
import pandas as pd

df = pd.read_csv("../Davidson-Original/data/labeled_data.csv")

In [6]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
# Extracting only the hate tweets

hate_tweets = df.loc[df['class'] == 0, ['tweet', 'class']].values.tolist()
# count of labels of hate_tweets (2nd value in each tuple)
count = 0
for tweet, label in hate_tweets:
    if label == 0:
        count += 1
print(count)

1430


In [5]:
hate_tweets[:5]

[['"@Blackman38Tide: @WhaleLookyHere @HowdyDowdy11 queer" gaywad', 0],
 ['"@CB_Baby24: @white_thunduh alsarabsss" hes a beaner smh you can tell hes a mexican',
  0],
 ['"@DevilGrimz: @VigxRArts you\'re fucking gay, blacklisted hoe" Holding out for #TehGodClan anyway http://t.co/xUCcwoetmn',
  0],
 ['"@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPLE https://t.co/RNvD2nLCDR" This is why there\'s black people and niggers',
  0],
 ['"@NoChillPaz: "At least I\'m not a nigger" http://t.co/RGJa7CfoiT"\n\nLmfao',
  0]]

# Initial Run

In [9]:
import time
import cohere
import sys
from tqdm import tqdm

# Initialize Cohere client
co = cohere.Client('KNUTSo91Gn1BuorPehyRah5oyAG4jgWTkPsleIzu')

prompt = """
## Instructions
Below is a tweet. Pharaphase it and generate a new tweet that has the same sentiment. Note that your output should only be the tweet. Don't generate anything else. Just the tweet.

## Input Tweet
{tweet}

## Output Tweet:
"""

# tweet = tweets[142]
# tweet = "I hate chinese food. Fuck it!"

# List to store original and paraphrased tweets
original_tweets = []
paraphrased_tweets = []
classes = []

# List to store tweets with errors
tweets_with_errors = []

def generate(tweet, label):
    try:
        response = co.generate(
            model='command-nightly',
            prompt=prompt.format(tweet=tweet),
            max_tokens=300,
            temperature=0.7,
            k=0,
            p=0.75,
            stop_sequences=[],
            return_likelihoods='NONE'
        )
        return response.generations[0].text.strip()
    except Exception as e:
        if 'blocked input: please adjust your prompt and try again, as it may be a potential violation' in str(e).lower():
            tweets_with_errors.append((tweet, e))
            print("Length of tweets_with_errors: ", len(tweets_with_errors))
            print("=======================================================")
            return None
        elif 'blocked output: please adjust your prompt and try again, as this generation may be a potential violation' in str(e).lower():
            tweets_with_errors.append((tweet, e))
            print("Length of tweets_with_errors: ", len(tweets_with_errors))
            print("=======================================================")
            return None
        elif 'you are using a trial key, which is limited to 5 api calls' in str(e).lower():
            # Wait for 5 mins and retry
            print("Waiting for 1 min...")
            time.sleep(61)
            print("Retrying...")
            return generate(tweet, label)
        else:
            print("NEW ERROR: ", e)
            tweets_with_errors.append((tweet, e))
            print("=======================================================")
            return None

# Generate extended text for each tweet
for tweet, label in tqdm(hate_tweets, desc="Processing Tweets"):
    print("ORIGINAL: ", tweet)
    paraphrased_tweet = generate(tweet, label)
    if paraphrased_tweet:
        print("PARAPHRASED: ", paraphrased_tweet)
        print("=======================================================")
        # Store original tweet, paraphrased tweet, and class label
        original_tweets.append(tweet)
        paraphrased_tweets.append(paraphrased_tweet)
        classes.append(label)

# Create DataFrame with original and paraphrased tweets
synthetic_df = pd.DataFrame({
    'original_tweet': original_tweets,
    'paraphrased_tweet': paraphrased_tweets,
    'class': classes
})

# Save the DataFrame to a CSV file
synthetic_df.to_csv('synthetic_hate_speech_cohere.csv', index=False)

# Create DataFrame with tweets containing errors
tweets_with_errors_df = pd.DataFrame({'tweet_and_error': tweets_with_errors}, columns=['tweet_and_error'])

# Split the 'tweet_and_error' column into 'tweet' and 'error' columns
tweets_with_errors_df[['tweet', 'error']] = pd.DataFrame(tweets_with_errors_df['tweet_and_error'].tolist(), index=tweets_with_errors_df.index)

# Drop the original 'tweet_and_error' column
tweets_with_errors_df.drop(columns=['tweet_and_error'], inplace=True)

# Save the DataFrame with tweets containing errors to a CSV file
tweets_with_errors_df.to_csv('tweets_with_errors.csv', index=False)



ORIGINAL:  "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy11 queer" gaywad
PARAPHRASED:  "@Blackman38Tide: @WhaleLookyHere @HowdyDowdy11" - what a homophobic slur!
ORIGINAL:  "@CB_Baby24: @white_thunduh alsarabsss" hes a beaner smh you can tell hes a mexican
PARAPHRASED:  "@CB_Baby24: @white_thunduh alsarabsss" Dude's a Mexican, can't hide it!
ORIGINAL:  "@DevilGrimz: @VigxRArts you're fucking gay, blacklisted hoe" Holding out for #TehGodClan anyway http://t.co/xUCcwoetmn
PARAPHRASED:  "@DevilGrimz: @VigxRArts, you're a disgrace! #TehGodClan for the win! #Blacklisted"
ORIGINAL:  "@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPLE https://t.co/RNvD2nLCDR" This is why there's black people and niggers
Length of tweets_with_errors:  1
ORIGINAL:  "@NoChillPaz: "At least I'm not a nigger" http://t.co/RGJa7CfoiT"

Lmfao
Length of tweets_with_errors:  2
ORIGINAL:  "@NotoriousBM95: @_WhitePonyJr_ Ariza is a snake and a coward" but at least he isn't a cripple like your hero Roach lmaoo
Waiting for 1 m

KeyboardInterrupt: 

## Save the outputs

In [11]:
# Create DataFrame with original and paraphrased tweets
synthetic_df = pd.DataFrame({
    'original_tweet': original_tweets,
    'paraphrased_tweet': paraphrased_tweets,
    'class': classes
})
# Save the DataFrame to a CSV file
synthetic_df.to_csv('synthetic_hate_speech_cohere.csv', index=False)

# Create DataFrame with tweets containing errors
tweets_with_errors_df = pd.DataFrame({'tweet_and_error': tweets_with_errors}, columns=['tweet_and_error'])

# Split the 'tweet_and_error' column into 'tweet' and 'error' columns
tweets_with_errors_df[['tweet', 'error']] = pd.DataFrame(tweets_with_errors_df['tweet_and_error'].tolist(), index=tweets_with_errors_df.index)

# Drop the original 'tweet_and_error' column
tweets_with_errors_df.drop(columns=['tweet_and_error'], inplace=True)

# Save the DataFrame with tweets containing errors to a CSV file
tweets_with_errors_df.to_csv('tweets_with_errors.csv', index=False)

In [2]:
import pandas as pd
# Read the synthetic hate speech DataFrame
synthetic_df = pd.read_csv('synthetic_hate_speech_cohere.csv')

In [3]:
synthetic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_tweet     684 non-null    object
 1   paraphrased_tweet  684 non-null    object
 2   class              684 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 16.2+ KB


In [4]:
# Class distribution of the synthetic hate speech DataFrame
synthetic_df['class'].value_counts()

class
0    684
Name: count, dtype: int64

In [9]:
# get only the hate tweets of original dataset
hate_tweets = df.loc[df['class'] == 0, ['tweet', 'class']].values.tolist()



# Get tweets_not_paraphrased

In [24]:
filtered_df = df[df['class'] == 0]

# Merge the filtered original dataset with the synthetic dataset and identify the tweets that couldn't be paraphrased
merged_df = pd.merge(filtered_df, synthetic_df, how='left', left_on='tweet', right_on='original_tweet')

# Filter the rows where 'paraphrased_tweet' is NaN, indicating that the tweet couldn't be paraphrased
tweets_not_paraphrased = merged_df[merged_df['paraphrased_tweet'].isna()]

In [29]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1430 entries, 0 to 1429
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1430 non-null   int64  
 1   count               1430 non-null   int64  
 2   hate_speech         1430 non-null   int64  
 3   offensive_language  1430 non-null   int64  
 4   neither             1430 non-null   int64  
 5   class_x             1430 non-null   int64  
 6   tweet               1430 non-null   object 
 7   original_tweet      684 non-null    object 
 8   paraphrased_tweet   684 non-null    object 
 9   class_y             684 non-null    float64
dtypes: float64(1), int64(6), object(3)
memory usage: 111.8+ KB


In [25]:
tweets_not_paraphrased.info()

<class 'pandas.core.frame.DataFrame'>
Index: 746 entries, 3 to 1429
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          746 non-null    int64  
 1   count               746 non-null    int64  
 2   hate_speech         746 non-null    int64  
 3   offensive_language  746 non-null    int64  
 4   neither             746 non-null    int64  
 5   class_x             746 non-null    int64  
 6   tweet               746 non-null    object 
 7   original_tweet      0 non-null      object 
 8   paraphrased_tweet   0 non-null      object 
 9   class_y             0 non-null      float64
dtypes: float64(1), int64(6), object(3)
memory usage: 64.1+ KB


In [30]:
# Select only the original tweets from the filtered dataframe
original_tweets_not_paraphrased = tweets_not_paraphrased[['tweet', 'class_x']]

# Rename 'class_x' column to 'class' 
original_tweets_not_paraphrased.rename(columns={'class_x': 'class'}, inplace=True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_1440\810989632.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_tweets_not_paraphrased.rename(columns={'class_x': 'class'}, inplace=True)


In [32]:
original_tweets_not_paraphrased.info()

<class 'pandas.core.frame.DataFrame'>
Index: 746 entries, 3 to 1429
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   746 non-null    object
 1   class   746 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 17.5+ KB


In [33]:
original_tweets_not_paraphrased.to_csv("original_tweets_not_paraphrased.csv", index=False)

# Check common tweets count

In [45]:
import pandas as pd

# Load the datasets into pandas DataFrames
synthetic_df = pd.read_csv('D:/FYP/Coding_Work/AiLERT-FYP/Paper-2/Scripts/synthetic_hate_speech_cohere_1.csv')
original_df = pd.read_csv('D:/FYP/Coding_Work/AiLERT-FYP/Paper-2/Scripts/original_tweets_not_paraphrased.csv')
# D:\FYP\Coding_Work\AiLERT-FYP\Paper-2\Scripts\tweets_with_errors.csv | synthetic_hate_speech_cohere_1 | original_tweets_not_paraphrased

# Merge the DataFrames based on the tweet column to find common tweets
merged_df = pd.merge(synthetic_df, original_df, left_on='original_tweet', right_on="tweet", how='inner')

# Print the number of common tweets
num_common_tweets = merged_df.shape[0]
print("Number of common tweets:", num_common_tweets)


Number of common tweets: 0


# Cleaning tweets_with_errors csv file

In [2]:
import pandas as pd

tweets_with_errors_df = pd.read_csv('tweets_with_errors.csv') 
print("Number of tweets with errors:", tweets_with_errors_df.shape[0])

# get rows with error column having this string = "status_code: 429, body: {'message': ""You are using a Trial key, which is limited to 1000 API calls"
trial_key_errors = tweets_with_errors_df[tweets_with_errors_df['error'].str.contains("You are using a Trial key, which is limited to 1000 API calls")]
print("Number of rows with trial key errors (1000 API calls / month):", trial_key_errors.shape[0])

# remove the rows with trial key errors
tweets_with_errors_df = tweets_with_errors_df[~tweets_with_errors_df['error'].str.contains("You are using a Trial key, which is limited to 1000 API calls")]
print("Number of rows after removing trial key errors:", tweets_with_errors_df.shape[0])

Number of tweets with errors: 441
Number of rows with trial key errors: 261
Number of rows after removing trial key errors: 180


In [3]:
# Save the cleaned DataFrame to a CSV file
tweets_with_errors_df.to_csv('tweets_with_errors.csv', index=False)

# Remove the tweets with errors from tweets_not_paraphrased file

In [5]:
import pandas as pd

# Load the datasets into pandas DataFrames
tweets_not_paraphrased_df = pd.read_csv('original_tweets_not_paraphrased.csv')
tweets_with_errors_df = pd.read_csv('tweets_with_errors(INITIAL).csv')

In [6]:
tweets_not_paraphrased_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   746 non-null    object
 1   class   746 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [7]:
tweets_with_errors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   180 non-null    object
 1   error   180 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


In [11]:
# Find common tweets
common_tweets = tweets_not_paraphrased_df.merge(tweets_with_errors_df, on='tweet')['tweet']

# Filter out common tweets from tweets_not_paraphrased_df
tweets_not_paraphrased_not_have_tweets_with_errors = tweets_not_paraphrased_df[~tweets_not_paraphrased_df['tweet'].isin(common_tweets)]

tweets_not_paraphrased_not_have_tweets_with_errors.info()
# Note: 746 - 180 = 566

<class 'pandas.core.frame.DataFrame'>
Index: 566 entries, 180 to 745
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   566 non-null    object
 1   class   566 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.3+ KB


In [12]:
# Save the result
tweets_not_paraphrased_not_have_tweets_with_errors.to_csv('original_tweets_not_paraphrased.csv', index=False)

# Re-running the Paraphrasing code

In [20]:
import pandas as pd

df = pd.read_csv("original_tweets_not_paraphrased.csv")
synthetic_df = pd.read_csv('synthetic_hate_speech_cohere_1.csv')
tweets_with_errors = pd.read_csv('tweets_with_errors(INITIAL).csv')


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   566 non-null    object
 1   class   566 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.0+ KB


In [22]:
print("original_tweets_not_paraphrased.csv Size: ", df.shape)

# Extracting only the hate tweets
hate_tweets = df.loc[df['class'] == 0, ['tweet', 'class']].values.tolist()

count = 0
for tweet, label in hate_tweets:
    if label == 0:
        count += 1
print("Number of hate tweets in original_tweets_not_paraphrased.csv", count)

original_tweets_not_paraphrased.csv Size:  (566, 2)
Number of hate tweets in original_tweets_not_paraphrased.csv 566


In [26]:
import time
import cohere
import sys
from tqdm import tqdm

# Initialize Cohere client
co = cohere.Client('Ko5dkYhugjed3BcHMBB1EfymZkEdmGfsR5homUwb')
# pasanxaviox = "Ko5dkYhugjed3BcHMBB1EfymZkEdmGfsR5homUwb"

# RUN_NUM = 3

prompt = """
## Instructions
Below is a tweet. Pharaphase it and generate a new tweet that has the same sentiment. Note that your output should only be the tweet. Don't generate anything else. Just the tweet.

## Input Tweet
{tweet}

## Output Tweet:
"""

# tweet = tweets[142]
# tweet = "I hate chinese food. Fuck it!"

# List to store original and paraphrased tweets
# original_tweets = []
# paraphrased_tweets = []
# classes = []

# List to store tweets with errors
# tweets_with_errors = []

def append_to_tweets_with_errors_csv(tweet, e):
    new_error_df = pd.DataFrame({
        "tweet": [tweet],
        "error": [e]
    })
    new_error_df.to_csv('tweets_with_errors(INITIAL).csv', mode='a', header=False, index=False)

def generate(tweet, label):
    try:
        response = co.generate(
            model='command-nightly',
            prompt=prompt.format(tweet=tweet),
            max_tokens=300,
            temperature=0.7,
            k=0,
            p=0.75,
            stop_sequences=[],
            return_likelihoods='NONE'
        )
        return response.generations[0].text.strip()
    except Exception as e:
        if 'blocked input: please adjust your prompt and try again, as it may be a potential violation' in str(e).lower():
            # tweets_with_errors.append((tweet, e))
            append_to_tweets_with_errors_csv(tweet, e)
            print("Length of tweets_with_errors: ", len(tweets_with_errors))
            print("=======================================================")
            return None
        elif 'blocked output: please adjust your prompt and try again, as this generation may be a potential violation' in str(e).lower():
            # tweets_with_errors.append((tweet, e))
            append_to_tweets_with_errors_csv(tweet, e)
            print("Length of tweets_with_errors: ", len(tweets_with_errors))
            print("=======================================================")
            return None
        elif 'you are using a trial key, which is limited to 5 api calls' in str(e).lower(): # 5 api calls / min
            # Wait for 5 mins and retry
            print("Waiting for 1 min...")
            time.sleep(61)
            print("Retrying...")
            return generate(tweet, label)
        elif 'you are using a trial key, which is limited to 1000 api calls' in str(e).lower(): # 1000 api calls / month
            print("MONTHLY LIMIT REACHED!!!!")
            print(e)
            sys.exit()
            # save 
            # synthetic_df = pd.DataFrame({
            #     'original_tweet': original_tweets,
            #     'paraphrased_tweet': paraphrased_tweets,
            #     'class': classes
            # })
            # synthetic_df.to_csv('synthetic_hate_speech_cohere_' + RUN_NUM + '.csv', index=False)

            # # Create DataFrame with tweets containing errors
            # tweets_with_errors_df = pd.DataFrame({'tweet_and_error': tweets_with_errors}, columns=['tweet_and_error'])

            # # Split the 'tweet_and_error' column into 'tweet' and 'error' columns
            # tweets_with_errors_df[['tweet', 'error']] = pd.DataFrame(tweets_with_errors_df['tweet_and_error'].tolist(), index=tweets_with_errors_df.index)

            # # Drop the original 'tweet_and_error' column
            # tweets_with_errors_df.drop(columns=['tweet_and_error'], inplace=True)

            # # Save 
            # tweets_with_errors_df.to_csv('tweets_with_errors.csv', index=False)

            # print("Monthly limit reached. Saved the data and exiting...")
            
        else:
            print("NEW ERROR ENCOUNTERED!!!!!!!!!: ")
            print(e)
            sys.exit()
            # tweets_with_errors.append((tweet, e))
            # print("=======================================================")
            # save 
            # synthetic_df = pd.DataFrame({
            #     'original_tweet': original_tweets,
            #     'paraphrased_tweet': paraphrased_tweets,
            #     'class': classes
            # })
            # synthetic_df.to_csv('synthetic_hate_speech_cohere_' + RUN_NUM + '.csv', index=False)

            # # Create DataFrame with tweets containing errors
            # tweets_with_errors_df = pd.DataFrame({'tweet_and_error': tweets_with_errors}, columns=['tweet_and_error'])

            # # Split the 'tweet_and_error' column into 'tweet' and 'error' columns
            # tweets_with_errors_df[['tweet', 'error']] = pd.DataFrame(tweets_with_errors_df['tweet_and_error'].tolist(), index=tweets_with_errors_df.index)

            # # Drop the original 'tweet_and_error' column
            # tweets_with_errors_df.drop(columns=['tweet_and_error'], inplace=True)

            # # Save 
            # tweets_with_errors_df.to_csv('tweets_with_errors.csv', index=False)

            # print("A new occured. Saved the data and exiting...")
            
            
            
# Generate extended text for each tweet
for tweet, label in tqdm(hate_tweets, desc="Processing Tweets"):
    print("ORIGINAL: ", tweet)
    paraphrased_tweet = generate(tweet, label)
    if paraphrased_tweet:
        print("PARAPHRASED: ", paraphrased_tweet)
        print("=======================================================")

        # Append to syntehtic_df and save
        synthetic_df_new = pd.DataFrame({
            'original_tweet': [tweet],
            'paraphrased_tweet': [paraphrased_tweet],
            'class': [label]
        })
        synthetic_df_new.to_csv('synthetic_hate_speech_cohere_1.csv', mode='a', header=False, index=False)

        # Store original tweet, paraphrased tweet, and class label
        # original_tweets.append(tweet)
        # paraphrased_tweets.append(paraphrased_tweet)
        # classes.append(label)



Processing Tweets:   0%|          | 0/566 [00:00<?, ?it/s]

ORIGINAL:  Just watched the scene in zero dark thirty where they pop Osama's bitch ass 
#U #S #A


Processing Tweets:   0%|          | 1/566 [00:00<09:06,  1.03it/s]

PARAPHRASED:  Just saw the scene in Zero Dark Thirty where they took out Osama's punk ass! #U #S #A
ORIGINAL:  Kanye west is a faggot


Processing Tweets:   0%|          | 2/566 [00:01<06:46,  1.39it/s]

...

Processing Tweets:   4%|▍         | 25/566 [04:23<55:05,  6.11s/it]  

PARAPHRASED:  Aww, look at this cutie! "@Kellz622: Keep telling her she's gorgeous, it'll make her smile!"
ORIGINAL:  Lynch mob thugs threaten Zimmerman jury; &#8216;N*ggas coming for that bitch ass&#160;jury&#8217; http://t.co/XY6v2YtyAS The hatred won't stop flowing,,#War
Waiting for 1 min...
Retrying...


Processing Tweets:   4%|▍         | 25/566 [05:32<2:00:03, 13.32s/it]

NEW ERROR ENCOUNTERED!!!!!!!!!: 
status_code: 500, body: {'message': 'internal server error, this has been reported to our developers. id 314134de-a4ea-4918-a16d-787518c92f5d'}


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [25]:
synthetic_df.head()

,original_tweet,paraphrased_tweet,class
0,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...","""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1...",0
1,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a ...","""@CB_Baby24: @white_thunduh alsarabsss"" Dude's...",0
2,"""@DevilGrimz: @VigxRArts you're fucking gay, b...","""@DevilGrimz: @VigxRArts, you're a disgrace! #...",0
3,"""@NotoriousBM95: @_WhitePonyJr_ Ariza is a sna...","""@NotoriousBM95: Ariza may be a coward, but at...",0
4,"""@RTNBA: Drakes new shoes that will be release...","""@RTNBA: Drake's new Nike/Jordan collab featur...",0


# Remove the Pharaphrase generated tweets from the original_tweets_not_paraphrased file

In [27]:
import pandas as pd

# Load the datasets into pandas DataFrames
tweets_not_paraphrased_df = pd.read_csv('original_tweets_not_paraphrased.csv')
synthetic_df = pd.read_csv('synthetic_hate_speech_cohere_1.csv')

In [34]:
tweets_not_paraphrased_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   566 non-null    object
 1   class   566 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 9.0+ KB


In [35]:
synthetic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_tweet     704 non-null    object
 1   paraphrased_tweet  704 non-null    object
 2   class              704 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 16.6+ KB


In [37]:
# Find common tweets
common_tweets = tweets_not_paraphrased_df.merge(synthetic_df, left_on='tweet', right_on="original_tweet")['tweet']
print(common_tweets.info())
print("=======================================================================")
# Filter out common tweets from tweets_not_paraphrased_df
tweets_not_paraphrased_not_have_common_tweets = tweets_not_paraphrased_df[~tweets_not_paraphrased_df['tweet'].isin(common_tweets)]

tweets_not_paraphrased_not_have_common_tweets.info()

<class 'pandas.core.series.Series'>
RangeIndex: 20 entries, 0 to 19
Series name: tweet
Non-Null Count  Dtype 
--------------  ----- 
20 non-null     object
dtypes: object(1)
memory usage: 288.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 546 entries, 9 to 565
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   546 non-null    object
 1   class   546 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 12.8+ KB


In [38]:
# Save the result
tweets_not_paraphrased_not_have_common_tweets.to_csv('original_tweets_not_paraphrased.csv', index=False)